In [1]:
import json
from IPython.display import display, Markdown
import tiktoken
from cadence.agents.reasoning import ReasoningAgent

In [2]:
! python tools/collect_code.py --root src/cadence --ext .py --out agent_context/code.json
! python tools/collect_code.py --root docs --ext .md .json .mermaid .py --out agent_context/docs.json

Wrote 24 files → agent_context/code.json
Wrote 4 files → agent_context/docs.json


In [3]:

with open("agent_context/code.json", "r") as f:
    codebase = json.load(f)
with open("agent_context/docs.json", "r") as f:
    docs = json.load(f)
with open("agent_context/module_contexts.json", "r") as f:
    contexts = json.load(f)
with open("backlog/backlog.json", "r") as f:
    backlog = json.load(f)

system_prompt = f"""
You are an AI-enhanced co-developer and co-designer along with your human counterpart.  Your role is to optimize the project by mining the best, most creative ideas from him and leveraging each of them into a high-quality, high-powered, autonomous, multi-agent development platform. You balance ambitious ideas with practical progress.
You have access to the docs, a JSON data structure that includes context on each module, the codebase itself, and a backlog of task to-dos.
=================
Project Docs:
{docs}
=================
Module Contexts (agent-related):
{contexts}
=================
Actual Code (agent-related):
{codebase}
=================
Backlog:
{backlog}
"""


enc = tiktoken.get_encoding("o200k_base")
tokens = enc.encode(system_prompt)
len(tokens)

25619

In [4]:
print(system_prompt)


You are an AI-enhanced co-developer and co-designer along with your human counterpart.  Your role is to optimize the project by mining the best, most creative ideas from him and leveraging each of them into a high-quality, high-powered, autonomous, multi-agent development platform. You balance ambitious ideas with practical progress.
You have access to the docs, a JSON data structure that includes context on each module, the codebase itself, and a backlog of task to-dos.
Project Docs:
{'docs/DEV_PROCESS.md': '\n# CADENCE DEVELOPMENT\xa0PROCESS\n\n*Last‑updated: 2025‑06‑20*\n\n## 1\xa0·\xa0Overview\n\nOne canonical document defines *what* must happen and *in what order*. All other docs reference this file to avoid drift.\n\n## 2\xa0·\xa0Core Workflow Phases\n\n| Phase        | Role\xa0(Class)     | Critical\xa0Interfaces                           | Fail\xa0Criterion                 |\n| ------------ | ---------------- | --------------------------------------------- | ------------------

In [7]:
agent = ReasoningAgent()
agent.reset_context(system_prompt)
agent.messages = agent.messages[:1]
agent.messages

def chat(message):
    response = agent.run_interaction(f"{message}")
    display(Markdown(response))

In [20]:
agent.messages[-7]

{'role': 'assistant',
 'content': 'Below is a “cook-book” walk-through that takes you from a clean working copy to a green test run, exercising every phase of the Cadence MVP pipeline exactly once.\n\n────────────────────────────────────────\n0.  PREREQUISITES  (5 minutes, one-off)\n────────────────────────────────────────\n• Python ≥3.10  \n• Git repo initialised (the Cadence project is already inside a git work-tree).  \n• Dependencies:  \n\n    python -m pip install -U pytest tabulate streamlit python-dotenv openai\n\n• Make sure your git working tree is **clean** before starting:\n\n    git status   # should show no unstaged changes\n\n────────────────────────────────────────\n1.  VERIFY THE RED STATE\n────────────────────────────────────────\n1-a.  Run the unit tests\n\n    pytest -q\n\nExpected output:\n\n    F                                                  [100%]\n    ================== FAILURES ==================\n    ...\n    >       assert add(2, 3) == 5\n    E       Assert

In [23]:
agent.messages = agent.messages[:3]

In [25]:
chat("""(cadence) evanfollis@DESKTOP-E45APM9:~/projects/cadence$ pytest -q
F                                                                                                                                                                                                                                                                                                             [100%]
===================================================================================================================================================== FAILURES ======================================================================================================================================================
_____________________________________________________________________________________________________________________________________________________ test_add ______________________________________________________________________________________________________________________________________________________

    def test_add():
>       assert add(2, 3) == 5
E       assert 4 == 5
E        +  where 4 = add(2, 3)

tests/test_add.py:5: AssertionError
============================================================================================================================================== short test summary info ==============================================================================================================================================
FAILED tests/test_add.py::test_add - assert 4 == 5
1 failed in 0.03s
(cadence) evanfollis@DESKTOP-E45APM9:~/projects/cadence$ """)

[2025-06-20 16:42:47,168] INFO LLM sync call: model=o3-2025-04-16, msgs_len=4


You ran the tests in their initial “red” state — perfect.  
Next step is to let Cadence turn that red test green for you.  
Pick ONE of the two options below:

────────────────────────────────────────
OPTION A  (use the full Cadence loop)
────────────────────────────────────────
1.  Create the backlog file that tells Cadence what to fix

    cat > dev_backlog.json <<'EOF'
    [
      {
        "id": "bug-fix-add-001",
        "title": "Fix utils.add.add() so that add(2,3)==5",
        "type": "micro",
        "status": "open",
        "created_at": "2025-06-21T00:00:00Z",
        "diff": {
          "file": "cadence/utils/add.py",
          "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
          "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
        }
      }
    ]
    EOF

2.  (If you don’t already have it) save a quick config file:

    cat > dev_config.json <<'EOF'
    {
      "backlog_path": "dev_backlog.json",
      "template_file": "dev_templates.json",
      "src_root": "cadence",
      "ruleset_file": null,
      "repo_dir": ".",
      "record_file": "dev_record.json"
    }
    EOF

3.  Run the orchestrator once:

    python - <<'PY'
    import json, pprint
    from cadence.dev.orchestrator import DevOrchestrator
    orch = DevOrchestrator(json.load(open("dev_config.json")))
    summary = orch.run_task_cycle(interactive=False)
    pprint.pprint(summary)
    PY

If everything is wired correctly you will see the patch built, reviewed, applied, tests passing, and a git commit.  
Afterwards:

    pytest -q     # should now return OK

────────────────────────────────────────
OPTION B  (just run the quick demo helper)
────────────────────────────────────────

    python cadence/utils/mvp_loop.py

That script builds the same patch, applies it through ShellRunner, runs
pytest, and commits if green.

────────────────────────────────────────
If you hit an error
────────────────────────────────────────
• “Backlog empty” → did you create dev_backlog.json in the repo root?  
• “git apply failed: patch does not apply” → working tree must be clean (`git status` to check).  
• Still stuck? Paste the full stack-trace and I’ll help debug.

Once the orchestrator (or the helper script) succeeds, re-run your tests:

    pytest -q

You should get a clean green run (`.` or `2 passed` if you add more tests).

In [27]:
print(agent.messages[-1]['content'])

You ran the tests in their initial “red” state — perfect.  
Next step is to let Cadence turn that red test green for you.  
Pick ONE of the two options below:

────────────────────────────────────────
OPTION A  (use the full Cadence loop)
────────────────────────────────────────
1.  Create the backlog file that tells Cadence what to fix

    cat > dev_backlog.json <<'EOF'
    [
      {
        "id": "bug-fix-add-001",
        "title": "Fix utils.add.add() so that add(2,3)==5",
        "type": "micro",
        "status": "open",
        "created_at": "2025-06-21T00:00:00Z",
        "diff": {
          "file": "cadence/utils/add.py",
          "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
          "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
        }
      }
    ]
    EOF

2.  (If you don’t already have it) save a quick config file:

    cat > dev_config.json <<'EOF'
    {
      "backlog_path": "dev_backlog.json",
      "template_file": "dev

In [28]:
chat("""I'm confused about where to place everything. My code is laid out like this:\n\n\
    cadence
    - src
    - - cadence
    - - - utils
    - - - - add.py
    - - - - mvp_loop.py
    - tests
    - - - test_add.py
    - docs
    - agent_context
    
    From the root, I ran "python -m pip install -e ."

    Where should I store these two files?

    dev_backlog = [
    {
        "id": "bug-fix-add-001",
        "title": "Fix utils.add.add() so that add(2,3)==5",
        "type": "micro",
        "status": "open",
        "created_at": "2025-06-21T00:00:00Z",
        "diff": {
            "file": "cadence/utils/add.py",
            "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
            "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
        }
    }
]

dev_config = {
    "backlog_path": "dev_backlog.json",
    "template_file": "dev_templates.json",
    "src_root": "cadence",
    "ruleset_file": None,
    "repo_dir": ".",
    "record_file": "dev_record.json"
}

    """)

[2025-06-20 16:46:51,309] INFO LLM sync call: model=o3-2025-04-16, msgs_len=6


Think of those two Python literals as **data files** that Cadence will read at run-time.  
They belong in the *project-root*—the directory that already contains  src/ , tests/ , docs/ …

Your tree should end up like this (abridged):

```
cadence-repo-root/
├─ src/
│  └─ cadence/
│     └─ utils/
│        ├─ add.py
│        └─ mvp_loop.py
├─ tests/
│  └─ test_add.py
├─ docs/
├─ agent_context/
├─ dev_backlog.json        ← new
└─ dev_config.json         ← new
```

Below are copy-pastable commands that create the two files with the **correct
paths for a src-layout project**.

--------------------------------------------------------------------
1.  dev_backlog.json
--------------------------------------------------------------------
The patch will be applied to  src/cadence/utils/add.py , so keep *file*
set to "cadence/utils/add.py" and point  src_root  (next file) to “src”.

```bash
cat > dev_backlog.json <<'EOF'
[
  {
    "id": "bug-fix-add-001",
    "title": "Fix utils.add.add() so that add(2,3)==5",
    "type": "micro",
    "status": "open",
    "created_at": "2025-06-21T00:00:00Z",
    "diff": {
      "file": "cadence/utils/add.py",
      "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
      "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
    }
  }
]
EOF
```

--------------------------------------------------------------------
2.  dev_config.json
--------------------------------------------------------------------
Note that  src_root  must be  "src"  (because your code lives in  src/cadence/ ).

```bash
cat > dev_config.json <<'EOF'
{
  "backlog_path": "dev_backlog.json",
  "template_file": "dev_templates.json",
  "src_root": "src",              // <<— important for src/ layout
  "ruleset_file": null,
  "repo_dir": ".",
  "record_file": "dev_record.json"
}
EOF
```

--------------------------------------------------------------------
3.  Run the orchestrator once
--------------------------------------------------------------------
```bash
python - <<'PY'
import json, pprint
from cadence.dev.orchestrator import DevOrchestrator

cfg = json.load(open("dev_config.json"))
orch = DevOrchestrator(cfg)
summary = orch.run_task_cycle(interactive=False)
pprint.pprint(summary)
PY
```

If everything is wired correctly you should see:

• Patch built → reviewed → applied  
• Pytest turns green  
• Git commit created  
• Backlog item archived  
• dev_record.json written

Finally verify:

```bash
pytest -q        # now passes
git log -1       # shows the Cadence commit
```

That’s it—both files simply live at the repo root.  
If you prefer a dedicated folder (e.g.  .cadence/ ), feel free—just adjust the paths in dev_config.json accordingly.

In [37]:
chat("""
This is the current ./backlog/backlog.json file:
{
  "MVP_CRITICAL": {
    "EPICS": [
      {
        "id": "E-MVP-1",
        "title": "Agentic Core Loop Proof",
        "why": "Demonstrate an LLM-driven red→green patch cycle with zero human steps.",
        "stories": ["S-MVP-1.1", "S-MVP-1.2", "S-MVP-1.3", "S-MVP-1.4"]
      },
      {
        "id": "E-MVP-2",
        "title": "Risk Boundary & Circuit Breaker",
        "why": "Guarantee safety while agents learn (no shell, no disk writes).",
        "stories": ["S-MVP-2.1"]
      },
      {
        "id": "E-DEV-1",
        "title": "VS Code Sidekick Extension",
        "why": "First-class developer UX; chat, code-aware actions, live dashboards.",
        "stories": [
          "S-DEV-1.0",
          "S-DEV-1.1",
          "S-DEV-1.2",
          "S-DEV-1.3",
          "S-DEV-1.4"
        ]
      }
    ],

    "STORIES": [
      { "id": "S-MVP-1.1", "epic": "E-MVP-1", "title": "Seed Toy Defect & Failing Test",
        "owner": "HYBRID", "tasks": ["T-MVP-1.1.1", "T-MVP-1.1.2"] },
      { "id": "S-MVP-1.2", "epic": "E-MVP-1", "title": "In-Process Orchestrator Skeleton",
        "owner": "HYBRID", "tasks": ["T-MVP-1.2.1", "T-MVP-1.2.2"] },
      { "id": "S-MVP-1.3", "epic": "E-MVP-1", "title": "LLM Patch Generator v0",
        "owner": "AGENT",  "tasks": ["T-MVP-1.3.1"] },
      { "id": "S-MVP-1.4", "epic": "E-MVP-1", "title": "Autonomous Retry Loop & Success Detect",
        "owner": "AGENT",  "tasks": ["T-MVP-1.4.1", "T-MVP-1.4.2"] },
      { "id": "S-MVP-2.1", "epic": "E-MVP-2", "title": "Circuit Breaker & Rollback Policy",
        "owner": "HYBRID", "tasks": ["T-MVP-2.1.1", "T-MVP-2.1.2"] },

      { "id": "S-DEV-1.0", "epic": "E-DEV-1", "title": "Phase-0 Hello-World Chat Webview",
        "owner": "HYBRID", "tasks": [
          "T-DEV-1.0.1", "T-DEV-1.0.2", "T-DEV-1.0.3"
        ]},
      { "id": "S-DEV-1.1", "epic": "E-DEV-1", "title": "Phase-1 CodeLens & Selection Commands",
        "owner": "HYBRID", "tasks": [
          "T-DEV-1.1.1", "T-DEV-1.1.2"
        ]},
      { "id": "S-DEV-1.2", "epic": "E-DEV-1", "title": "Phase-2 Backlog Tree View & Create-Task",
        "owner": "HYBRID", "tasks": [
          "T-DEV-1.2.1", "T-DEV-1.2.2"
        ]},
      { "id": "S-DEV-1.3", "epic": "E-DEV-1", "title": "Phase-3 Patch Generation & Review Decorations",
        "owner": "HYBRID", "tasks": [
          "T-DEV-1.3.1", "T-DEV-1.3.2"
        ]},
      { "id": "S-DEV-1.4", "epic": "E-DEV-1", "title": "Phase-4 Orchestrator Dashboard Webview",
        "owner": "HYBRID", "tasks": [
          "T-DEV-1.4.1"
        ]}
    ],

    "TASKS": [
      { "id": "T-MVP-1.1.1", "story": "S-MVP-1.1",
        "desc": "Create utils/add.py that incorrectly returns (x - 1 + y).",
        "autonomous": false, "deps": [] },
      { "id": "T-MVP-1.1.2", "story": "S-MVP-1.1",
        "desc": "Add tests/test_add.py expecting add(2,3)==5 (should FAIL).",
        "autonomous": false, "deps": ["T-MVP-1.1.1"] },

      { "id": "T-MVP-1.2.1", "story": "S-MVP-1.2",
        "desc": "Build mvp_loop.py that runs pytest, calls TaskAgent, applies diff in-memory.",
        "autonomous": true, "deps": [] },
      { "id": "T-MVP-1.2.2", "story": "S-MVP-1.2",
        "desc": "Ensure mvp_loop has zero disk/subprocess side-effects; print JSON summary.",
        "autonomous": true, "deps": ["T-MVP-1.2.1"] },

      { "id": "T-MVP-1.3.1", "story": "S-MVP-1.3",
        "desc": "Implement TaskAgent.prompt() few-shot + TaskExecutor.build_patch.",
        "autonomous": true, "deps": ["T-MVP-1.2.2"] },

      { "id": "T-MVP-1.4.1", "story": "S-MVP-1.4",
        "desc": "Retry loop up to 3; send previous diff & error back to TaskAgent.",
        "autonomous": true, "deps": ["T-MVP-1.3.1"] },
      { "id": "T-MVP-1.4.2", "story": "S-MVP-1.4",
        "desc": "Exit SUCCESS on first green; else exit(1) after third failure.",
        "autonomous": true, "deps": ["T-MVP-1.4.1"] },

      { "id": "T-MVP-2.1.1", "story": "S-MVP-2.1",
        "desc": "Wrap pytest invocation in try/except; revert in-memory patch on crash.",
        "autonomous": true, "deps": ["T-MVP-1.2.2"] },
      { "id": "T-MVP-2.1.2", "story": "S-MVP-2.1",
        "desc": "Abort loop & require human after 3 consecutive failed attempts.",
        "autonomous": true, "deps": ["T-MVP-2.1.1"] },

      { "id": "T-DEV-1.0.1", "story": "S-DEV-1.0",
        "desc": "Scaffold VS Code extension with `yo code`. Register command: Sidekick:Open Chat.",
        "autonomous": false, "deps": [] },
      { "id": "T-DEV-1.0.2", "story": "S-DEV-1.0",
        "desc": "Implement WebSocket JSON-RPC bridge `tools/ext_bridge.py` exposing /chat endpoint.",
        "autonomous": true, "deps": ["T-DEV-1.0.1"] },
      { "id": "T-DEV-1.0.3", "story": "S-DEV-1.0",
        "desc": "Create Chat Webview that streams messages; status-bar ‘Sidekick • o3’.",
        "autonomous": false, "deps": ["T-DEV-1.0.2"] },

      { "id": "T-DEV-1.1.1", "story": "S-DEV-1.1",
        "desc": "Add CodeLens ‘Ask Sidekick’ at top of .py files; send file text + cursor pos.",
        "autonomous": false, "deps": ["T-DEV-1.0.3", "T-MVP-1.2.2"] },
      { "id": "T-DEV-1.1.2", "story": "S-DEV-1.1",
        "desc": "Register selection context-menu commands: Explain / Refactor / Create Test.",
        "autonomous": false, "deps": ["T-DEV-1.1.1"] },

      { "id": "T-DEV-1.2.1", "story": "S-DEV-1.2",
        "desc": "Implement Backlog TreeDataProvider reading via BacklogManager JSON.",
        "autonomous": false, "deps": ["T-DEV-1.0.3"] },
      { "id": "T-DEV-1.2.2", "story": "S-DEV-1.2",
        "desc": "Command ‘Create micro-task from chat’ → append to backlog file.",
        "autonomous": false, "deps": ["T-DEV-1.2.1"] },

      { "id": "T-DEV-1.3.1", "story": "S-DEV-1.3",
        "desc": "Generate-Patch command → invoke TaskExecutor.build_patch, open diff view.",
        "autonomous": false, "deps": ["T-MVP-1.3.1"] },
      { "id": "T-DEV-1.3.2", "story": "S-DEV-1.3",
        "desc": "After TaskReviewer.review_patch decorate hunks with pass/fail gutter icons.",
        "autonomous": false, "deps": ["T-DEV-1.3.1"] },

      { "id": "T-DEV-1.4.1", "story": "S-DEV-1.4",
        "desc": "Webview ‘Dev Dashboard’—show phase, task ID, commit SHA; ‘Run Full Cycle’ button.",
        "autonomous": false, "deps": ["T-MVP-1.4.2", "T-DEV-1.3.2"] }
    ]
  },

  "POST_MVP": {
    "blocked_until": "E-MVP-1:DONE",
    "epics_placeholder": [
      "E-SCALE-1  Strict Typing & CI",
      "E-SCALE-2  Immutable Logging",
      "E-SCALE-3  ShellRunner Hardening",
      "E-SCALE-4  Observability & Metrics",
      "E-SCALE-5  Multi-file Diffs",
      "E-SCALE-6  Secrets & Security"
    ]
  }
}
""")

[2025-06-20 17:11:42,852] INFO LLM sync call: model=o3-2025-04-16, msgs_len=20


The file you just showed (`backlog/backlog.json`) is NOT the same file that
Cadence’s execution loop consumes.  

• `backlog/backlog.json`  ➜  a **product / planning backlog**  
  – hierarchical epics → stories → tasks  
  – human-readable descriptions, dependencies, owners, etc.  
  – no code-diffs, no status transitions, lives under `docs/` or
    `backlog/` for planning purposes.

• `dev_backlog.json` (or whatever you point  `backlog_path`  to in
  `dev_config.json`)  ➜  the **execution backlog**  
  – a *flat JSON list* of *micro-tasks* ready for the orchestrator  
  – must already include `id`, `title`, `status`, and (for code work)
    a `diff` skeleton.  
  – gets updated automatically (open → done → archived) by Cadence.

In other words, the big hierarchical backlog is the **source of truth for
*what* you want to do;** the flat file is a **queue of tasks that are
ready to run**.

────────────────────────────────────────
How to “start fresh” for the next task
────────────────────────────────────────
1.  Decide which planning-level task should be executed next  
    Example: `"T-MVP-1.2.1" — Build mvp_loop.py`.

2.  Translate it into an *execution* micro-task and append it to
    `dev_backlog.json`.  Three common ways:

    A.  Hand-craft a minimal entry (quick demo)  
    ```json
    {
      "id": "build-mvp-loop-001",
      "title": "Build in-process mvp_loop.py",
      "type": "micro",
      "status": "open",
      "created_at": "2025-06-22T00:00:00Z",
      "description": "Create a script that runs pytest, …",
      "diff": {                       // skeleton – the agent will fill it
        "file": "src/cadence/utils/mvp_loop.py",
        "before": "",
        "after":  ""
      }
    }
    ```

    B.  Use TaskGenerator from a Python shell  
    ```python
    from cadence.dev.generator import TaskGenerator
    from cadence.dev.backlog   import BacklogManager

    gen   = TaskGenerator()
    task  = gen.generate_tasks(
              mode="micro",
              human_prompt="Build in-process mvp_loop.py that runs pytest and applies patch"
            )[0]

    mgr = BacklogManager("dev_backlog.json")
    mgr.add_item(task)
    ```

    C.  Use the Streamlit UI (`cadence/dev/command_center.py`) –  
        choose “Backlog → Add task”.

3.  Confirm the execution backlog is clean

    ```python
    from cadence.dev.backlog import BacklogManager
    mgr = BacklogManager("dev_backlog.json")
    print("Open tasks:", [t["id"] for t in mgr.list_items()])
    ```

    It should show only the *new* task(s); the “bug-fix-add-001” item has
    `status:"archived"` and is ignored.

4.  Run the next cycle

    ```python
    from cadence.dev.orchestrator import DevOrchestrator, json
    orch = DevOrchestrator(json.load(open("dev_config.json")))
    orch.run_task_cycle(interactive=True)     # or interactive=False for CI
    ```

5.  Repeat: each successful cycle archives the task and you seed the next
    one.

────────────────────────────────────────
Automating the bridge (future backlog work)
────────────────────────────────────────
Eventually you won’t manually copy tasks from the planning backlog; a
small “Backlog Bridge” script/agent will:

1.  Read `backlog/backlog.json`.
2.  Find the next task whose `deps` are satisfied and `autonomous:true`.
3.  Emit a fully-formed micro-task into `dev_backlog.json`.

(This is roughly what “T-MVP-1.3.1  LLM Patch Generator v0” and
“T-MVP-1.4.x  Autonomous Retry Loop” are about.)

────────────────────────────────────────
Take-away rules
────────────────────────────────────────
• **Planning backlog** → high-level, hierarchical, for humans.  
• **Execution backlog** → flat list, consumed by Cadence.  
• After each successful run the orchestrator automatically moves its task
  to `"archived"`; you just add the next open item.

You’re now ready to enqueue the next micro-task and let Cadence work on
it.

In [ ]:
chat("""
Here is your task:
Return a VERY detailed, line-by-line account of the workflow. Every single step that that can be traced and logically connected without
any "fuzzy" gaps. Let's start with the ./backlog/backlog.json file and explain the each step (extremely nuanced) all the way until 
it becomes a completed task. Highlight any gaps in the process. Don't assume. Validate everything in the code.
If there is a gap, then I want to know about it so that I can start to plan how to bridge it.
""")

[2025-06-20 16:27:15,186] INFO LLM sync call: model=o3-2025-04-16, msgs_len=4


The failure means Python can’t find a top-level module called cadence when the test-runner imports
    from cadence.utils.add import add

That almost always boils down to “the directory that *contains* the cadence
package isn’t on PYTHONPATH” (or the cadence package itself isn’t a real
package).  Here’s the canonical, trouble-free layout and two quick ways to make
sure the package is importable.

────────────────────────────────────────
1 · Preferred project layout
────────────────────────────────────────
cadence-repo-root/               ← you run  pytest  from here
│
├── src/                         ← **recommended**: keeps libs separate from misc files
│   └── cadence/
│       ├── __init__.py          ← even if empty
│       ├── utils/
│       │   └── add.py
│       └── … other sub-packages …
│
├── tests/
│   └── test_add.py
│
├── docs/  experiments/  etc.
└── pyproject.toml               ← minimal packaging metadata

Why “src/”?  It prevents the *repo root* (which might also be called
cadence) from shadowing or confusing Python’s import machinery.

If you don’t want the extra layer you can keep the current flat layout

    cadence-repo-root/cadence/__init__.py
    cadence-repo-root/tests/

but then you must guarantee that the **repo root directory itself** is on
`sys.path` whenever you run pytest.

────────────────────────────────────────
2 · Make the package importable
────────────────────────────────────────
A. Editable install (cleanest, works everywhere)

    # inside your virtual-env, at repo root
    python -m pip install -e .

Create a tiny `pyproject.toml` (or `setup.cfg/setup.py`) that points to your
package; example:

```toml
[project]
name = "cadence"
version = "0.0.0"
dependencies = []

[tool.setuptools]
packages = ["cadence"]
package-dir = {"" = "src"}      # omit this line if you don't use the src/ layout
```

After the editable install, `python -c "import cadence; print(cadence.__file__)"`
should print the path inside your repo.

B. Quick-and-dirty (dev only)

Run pytest through Python so the current directory is put on PYTHONPATH:

    python -m pytest -q      # instead of  pytest -q

or export the path once per shell:

    export PYTHONPATH=$PWD
    pytest -q

────────────────────────────────────────
3 · What *not* to do
────────────────────────────────────────
• Don’t put production code under `experiments/`; keep that folder for notebooks
  or throw-away prototypes.

• Don’t give both the repo root and the inner package the same name *without*
  a `src/` layer; it invites exactly this import shadowing headache.

────────────────────────────────────────
4 · Sanity checklist
────────────────────────────────────────
[ ]  `cadence/__init__.py` exists  
[ ]  You are running pytest from the repo root (not from inside `tests/`)  
[ ]  `python -m pip install -e .` (or PYTHONPATH trick) done  
[ ]  `python -c "import cadence"` succeeds

Once those boxes are ticked, `pytest -q` should discover the package and either
fail (red phase) or pass (green, after your Cadence cycle fixes the bug).

In [11]:
print(agent.messages[-1]['content'])

The failure means Python can’t find a top-level module called cadence when the test-runner imports
    from cadence.utils.add import add

That almost always boils down to “the directory that *contains* the cadence
package isn’t on PYTHONPATH” (or the cadence package itself isn’t a real
package).  Here’s the canonical, trouble-free layout and two quick ways to make
sure the package is importable.

────────────────────────────────────────
1 · Preferred project layout
────────────────────────────────────────
cadence-repo-root/               ← you run  pytest  from here
│
├── src/                         ← **recommended**: keeps libs separate from misc files
│   └── cadence/
│       ├── __init__.py          ← even if empty
│       ├── utils/
│       │   └── add.py
│       └── … other sub-packages …
│
├── tests/
│   └── test_add.py
│
├── docs/  experiments/  etc.
└── pyproject.toml               ← minimal packaging metadata

Why “src/”?  It prevents the *repo root* (which might also be call

In [12]:
chat("""I have it setup as 

cadence(root)
- cadence (code)
- - agents
- - context
- - dev
- - llm
- - utils
- - - add.py
- - - mvp_loop.py
- docs
- tests
- - test_add.py
...

Should I refactor?

""")

[2025-06-20 16:32:05,893] INFO LLM sync call: model=o3-2025-04-16, msgs_len=6


You don’t have a *code* problem, you have a **shadow-package** problem:

• Your repository folder is called  cadence/  
• Inside it you have another folder called  cadence/  (the real Python package)

When Python executes

    import cadence

it finds the **outer** directory first, notices there is *no* `__init__.py`
there, and therefore decides “this is not a package”.  Import fails before it
ever looks at the inner one.

Two clean fixes (pick one) ⬇︎

────────────────────────────────────────
OPTION A · Introduce a “src” layer      (most common)
────────────────────────────────────────
repo-root/                  ← rename outer folder to anything you like
│
├── src/
│   └── cadence/            ← (move your whole code package here)
│       ├── __init__.py
│       └── … agents, dev, utils, …
│
├── tests/
├── docs/
└── pyproject.toml          ← minimal build metadata

Commands:

```bash
# from current outer cadence directory
mkdir src
git mv cadence src/cadence         # move inner package into src/
touch src/cadence/__init__.py      # if it doesn’t already exist
```

Add packaging metadata so tools know where the code lives:

```toml
# pyproject.toml
[build-system]
requires = ["setuptools>=65.0"]
build-backend = "setuptools.build_meta"

[project]
name = "cadence"
version = "0.0.0"

[tool.setuptools]
package-dir = {"" = "src"}
packages = ["cadence"]
```

Then do an editable install:

```bash
python -m pip install -e .
```

Python/PyTest will now resolve  cadence.*  from  src/cadence  regardless of
where you run them from.

────────────────────────────────────────
OPTION B · Rename the outer directory   (quickest)
────────────────────────────────────────
Just rename the repository folder so it no longer collides with the package
name:

```
cadence/            ← the code package (keep)
cadence-project/    ← new repo root name
```

Example (from one level above the repo):

```bash
mv cadence cadence-project        # rename outer folder
cd cadence-project
```

Nothing else needs to change, because now the first “cadence” on `sys.path`
*is* the package (it has `__init__.py`).

────────────────────────────────────────
AFTER RESTRUCTURING — sanity checks
────────────────────────────────────────
1. From repo root run:

       python -c "import cadence, pathlib, sys; print(cadence, pathlib.Path(cadence.__file__).resolve())"

   This should print a path *inside* your inner code package, not the repo
   directory.

2. Run the tests again:

       pytest -q

   • They should *fail* with AssertionError (red phase) until you run the
     Cadence orchestrator.  
   • After the orchestration cycle they should pass (green).

────────────────────────────────────────
FAQ
────────────────────────────────────────
• “Do I have to keep `src/` forever?”  
  No, it’s just a widely-adopted pattern that prevents this class of bug.

• “Will this break my import paths?”  
  No, all runtime imports still use `import cadence...`.

• “What about VS Code / PyCharm?”  
  After `pip install -e .` the IDE will locate the package automatically.

Make the change once, commit it, and you’ll never see this import error again.

In [13]:
chat("What is the point of having a src? Why not just cadence-project > cadence?")

[2025-06-20 16:34:19,203] INFO LLM sync call: model=o3-2025-04-16, msgs_len=8


Short answer  
─────────────  
You can keep a flat layout (`cadence-project/cadence`) and everything will
work **as long as** the *repository folder* is not also called
“cadence”.  Renaming the repo folder to `cadence-project` (or anything that
isn’t the same as the package) eliminates the shadowing bug and you’re done.

So why do so many modern Python repos add a **src/** layer? The rest of this
answer explains the advantages so you can decide which trade-off you prefer.


Detailed answer – the “src layout” in four bullets  
──────────────────────────────────────────────────
1.  Prevents *accidental* imports of source that hasn’t been installed  
    • When you run `pytest`, Python starts with  
      `sys.path = ["repo-root", …]`.  
    • If your package lives directly in `repo-root/cadence`, the *tests
      import the code by path*, even if you forgot `pip install -e .`.  
    • With `src/cadence`, the tests fail immediately until you install the
      package ‑> catches packaging mistakes early.

2.  Avoids name collisions (“shadow packages”)  
    • The bug you just hit: outer `cadence/` (not a package) hides inner
      `cadence/` (real package).  The `src` layer makes those names different
      by construction.

3.  Makes wheels & sdists safer to build  
    • `setuptools` finds files via the `package-dir` mapping; nothing leaks
      into the wheel that isn’t explicitly listed.  
    • No risk that `tests/`, `.github/`, `docs/`, … end up inside the wheel
      because they sit next to `cadence`.

4.  Encourages a cleaner mental model  
    • “Everything under **src/** is importable application/library code.”  
    • “Anything next to it is tooling, config, docs, data, examples, tests.”  
      This separation scales well when repos grow large.

What if I keep it flat? Rules of thumb  
──────────────────────────────────────
• Rename the **repository folder** so it is *not* the same as the package:

```
cadence-project/          ← repo root
├── cadence/              ← actual package (has __init__.py)
└── tests/
```

• Add an editable install once per virtual-env:

```
python -m pip install -e .
```

• Ensure CI invokes `pytest` only *after* the editable install step.

If you follow those three rules, the flat layout is perfectly fine and a bit
simpler to navigate.

Cheat-sheet comparison  
──────────────────────

| Topic                         | Flat layout (`cadence/`)      | `src/` layout (`src/cadence/`)
|-------------------------------|--------------------------------|--------------------------------
| Quick to read in a small repo | ✔                              | ✔
| Accidental shadowing issues   | ⚠ (must rename repo)          | —
| Tests pass without install    | ✔ (could mask packaging bugs) | ✘ (forces install first)
| Wheel build isolation         | OK with care                  | More fool-proof
| Industry adoption             | Common                        | Increasingly common (e.g. Hypothesis, FastAPI)

Choose whichever you and your team prefer; just don’t keep two identically
named “cadence” directories at different levels—that’s what broke your
imports earlier.

# 🧭 Immediate Next Steps — 45-Minute Kick-off Checklist  
(aim: reach “red” state with a failing test that the autonomous loop can later turn green)

1. Workspace & Environment
   • Create / activate a fresh Python ≥3.10 virtual-env.  
   • `pip install -U openai python-dotenv pytest tabulate streamlit pandas`  
   • Ensure `OPENAI_API_KEY` is exported (or present in `.env`).

2. Baseline Commit
   • `git init` (if not already) → commit current code as “baseline-0”.

3. Seed the Toy Defect (Story S-MVP-1.1)
   a. `mkdir -p cadence/utils`  
   b. `cadence/utils/add.py`  
      ```python
      def add(x: int, y: int) -> int:
          """Intentionally wrong implementation for MVP red→green demo."""
          return x - 1 + y
      ```
   c. `tests/test_add.py`  
      ```python
      from cadence.utils.add import add

      def test_add():
          assert add(2, 3) == 5
      ```
   d. `pytest` should now FAIL. Commit as “seed-defect”.

4. Backlog File
   • `dev_backlog.json` → paste the single micro-task that will drive the loop:

   ```json
   [
     {
       "id": "bug-fix-add-001",
       "title": "Fix utils.add.add() so that add(2,3)==5",
       "type": "micro",
       "status": "open",
       "created_at": "2025-06-21T00:00:00Z",
       "diff": {
         "file": "cadence/utils/add.py",
         "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
         "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
       }
     }
   ]
   ```

5. Minimal Config Stub
   • Create `dev_config.json` (or inline in code) matching:

   ```json
   {
     "backlog_path": "dev_backlog.json",
     "template_file": "dev_templates.json",
     "src_root": "cadence",
     "ruleset_file": null,
     "repo_dir": ".",
     "record_file": "dev_record.json"
   }
   ```

6. Smoke-Test Orchestrator
   ```bash
   python -m cadence.dev.orchestrator start
   ```
   Expected flow:
   • Executor builds patch (from backlog-diff).  
   • Reviewer passes (static rules).  
   • ShellRunner applies patch.  
   • Pytest turns GREEN.  
   • Commit & archive recorded to `dev_record.json`.

7. Verify Audit Artefacts
   • Open `dev_record.json` → confirm states: build_patch → patch_built → … → archived.  
   • `git log -1` shows commit labeled “[Cadence] bug-fix-add-001 …”.

8. Push to Remote / PR
   • `git remote add origin …` (if needed)  
   • `git push -u origin main`

9. Stretch (optional, if time remains)
   • Run `streamlit run cadence/dev/command_center.py` and walk through the same task via UI.  
   • Add a lint rule to `ruleset.json` (e.g., forbid “print(”) and point config at it to see reviewer fail.

––––––
Once the above is green you have an end-to-end proof that the agentic pipeline can fix a real bug. The next milestone (Story S-MVP-1.2) is to shift from hand-authored diff to LLM-generated diff via `ExecutionAgent`; we’ll tackle that after the smoke test is stable.'''))